In [1]:
# Imports
import keras
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os, glob, random, sys, math, cv2
from sklearn.model_selection import train_test_split
from skimage import transform
from tqdm import tqdm

from model import Model

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# Read image categories
ROOT_DIR = os.getcwd()
IMAGE_DIR = os.path.join(ROOT_DIR, 'images')
CATEGORIES = [folder for folder in os.listdir(IMAGE_DIR) 
               if os.path.isdir(os.path.join(IMAGE_DIR, folder))]
CATEGORIES.sort() # Alphabetical order
NUM_CLASSES = len(CATEGORIES)
print(CATEGORIES)

['aeschynanthus_parvifolius', 'bougainvillea_glabra', 'costus_woodsonii_maas', 'hedychium', 'hibiscus_rosa_sinensis', 'ixora_congesta', 'jasminum_sambac', 'lycoris', 'papilionanthe_miss_joaqium', 'plumeria']


In [3]:
IMAGE_DIM = (512, 512,)

data_present = False

if os.path.exists('data') and len(glob.glob('data')) > 0:
    x_train = np.load('data/train_images.npy')
    y_train = np.load('data/train_labels.npy')
    data_present = True
    
else:
    # Read images and labels into arrays
    images = []
    labels = []
    for label, category in enumerate(CATEGORIES):
        print("Processing {} images...".format(category))
        folder_path = os.path.join(IMAGE_DIR, category)
        # Read images in subfolder
        for image_file_path in tqdm(glob.glob(folder_path + "/*.jpg")):
            image_data = cv2.imread(image_file_path)
            if image_data is None:
                print("Faulty image {}; Ignoring...".format(image_file_path))
                continue
            # Resize to constant dimensions
            image_data = transform.resize(image_data, IMAGE_DIM)
            images.append(image_data)
            labels.append(label)

    # Change labels to one-hot vector of length NUM_CLASSES instead of integers
    labels = keras.utils.to_categorical(labels)

    print("Read {} images and {} labels.".format(len(images), len(labels))) # Should be same number

In [4]:
if not data_present:
    # Split into training and test sets
    train_test_split_ratio = 0.9 # Proportion in train set
    x_train, x_test, y_train, y_test = train_test_split(images, labels, train_size = train_test_split_ratio)
    print("{} training images, {} testing images".format(len(x_train), len(x_test)))

    # Convert to numpy arrays
    x_train, x_test, y_train, y_test = np.array(x_train), np.array(x_test), np.array(y_train), np.array(y_test)

In [5]:
if not data_present:
    # Save arrays to file
    if not os.path.exists('data'):
        os.makedirs('data')
    np.save('data/train_images', x_train)
    np.save('data/train_labels', y_train)
    np.save('data/test_images', x_test)
    np.save('data/test_labels', y_test)

In [6]:
# Image data augmentation to increase effective dataset size
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255, # Rescale to range 0..1
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='wrap', # I.e. tiling
)

In [7]:
# Model
MODEL_NAME = "densenet"
model = Model.getCompiledModel(MODEL_NAME, IMAGE_DIM, NUM_CLASSES)

# Callback to save model weights
model_checkpoint = keras.callbacks.ModelCheckpoint(MODEL_NAME + '_weights.{epoch:d}.h5', verbose=1)

In [8]:
# Load weights if they exist
last_epoch = 0
# model.load_weights('{}_weights.{:d}.h5'.format(MODEL_NAME, last_epoch))

In [ ]:
# Train model

# Hyper parameters
BATCH_SIZE = 8
EPOCHS = 500
STEPS_PER_EPOCH = len(x_train) // BATCH_SIZE

model.fit_generator(
    datagen.flow(x_train, y_train, batch_size = BATCH_SIZE),
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs = EPOCHS,
    callbacks = [model_checkpoint],
    initial_epoch = last_epoch
)

Epoch 1/500
101/101 [==============================] - 161s 2s/step - loss: 2.2853 - acc: 0.2909

Epoch 00001: saving model to densenet_weights.1.h5
Epoch 2/500
101/101 [==============================] - 148s 1s/step - loss: 2.0855 - acc: 0.2946

Epoch 00002: saving model to densenet_weights.2.h5
Epoch 3/500
 48/101 [=============>................] - ETA: 1:17 - loss: 1.9246 - acc: 0.3438